## Bibliotecas

In [1]:
import pickle
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

%matplotlib inline

## Análise Exploratória

In [2]:
# Datasets
anime = pd.read_csv('anime.csv') # Dataframe sobre programas de entreterimento do tipo anime ou animações japônesas.
usuario = pd.read_csv('rating.csv') # Dataframe comportamento de usuários da plataforma em avaliar animes.

In [3]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
anime.info() # info das colunas e seus tipos do dataset anime.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [5]:
anime.isnull().sum() # verificar o número de valores nulos.

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

Identificamos uma quantidade de 317 registros com valores nulos, neste caso vamos dropar esses registros por não saber a natureza que levou a ausência de dados que poderia ser n motivos. 

Dentre eles podemos citar erro humano, erro sistémico, poder ser animes que não lançaram ainda...

In [6]:
anime.dropna(inplace=True) #Drop dos valores nulos no dataser anime.

vamos utilizar em grande parte a coluna *genre* para realizar as recomendações por base nos generos de cada desenho japônes possui e identificar padrões., 

O metodo para conseguir isso de forma eficiente e que uma máquina consiga entender é pelo método *one hot encoding*, na lib pandas é chamada de *get_dummies()*, que realiza uma transposição binária em 0 e 1 de valores categoricos de uma coluna para uma nova coluna como no exemplo abaixo:

In [7]:
one_hot_genre = anime['genre'].str.get_dummies(',')

In [8]:
one_hot_genre

,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,Shoujo,Shounen,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Abaixo identificamos um problema: a base possui valores duplicados depois do método one hot encoding por causa de grafia de alguns valores, pois eles apresentam espaço em branco ou leading whitespace.

Temos que resolver isso, a melhor forma é utilizando funções de regex.

In [9]:
one_hot_genre.columns

Index([' Adventure', ' Cars', ' Comedy', ' Dementia', ' Demons', ' Drama',
       ' Ecchi', ' Fantasy', ' Game', ' Harem', ' Hentai', ' Historical',
       ' Horror', ' Josei', ' Kids', ' Magic', ' Martial Arts', ' Mecha',
       ' Military', ' Music', ' Mystery', ' Parody', ' Police',
       ' Psychological', ' Romance', ' Samurai', ' School', ' Sci-Fi',
       ' Seinen', ' Shoujo', ' Shoujo Ai', ' Shounen', ' Shounen Ai',
       ' Slice of Life', ' Space', ' Sports', ' Super Power', ' Supernatural',
       ' Thriller', ' Vampire', ' Yaoi', ' Yuri', 'Action', 'Adventure',
       'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy',
       'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids',
       'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery',
       'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School',
       'Sci-Fi', 'Seinen', 'Shoujo', 'Shounen', 'Slice of Life', 'Space',
       'Sports', 'Super Power', 'Supernatural'

In [10]:
anime = anime.assign(genre=np.core.defchararray.split(anime.genre.values.astype(str), ',')) #Criamos um array com o conteúdo do genre, ficará mais facil de manipular dps

In [11]:
# Esse for irá vasculhar a coluna genre para eliminar espaços em branco antes e depois da palavra.
genre_lista = [] # array auxiliar.
for i in anime['genre']: # For que percorre todos os registros da coluna genre.
    array_raw = np.array(i) #Transforma o conteúdo em array numpy.
    for a in range(0,len(array_raw)) : # Esse for percorre o array do conteúdo.
        array_raw[a] = array_raw[a].strip() # Tira os Leading Whitesapces.
    array_final = ','.join([str(x) for x in array_raw]) # Transforma o array em string novamente.
    genre_lista.append(array_final) # realiza o apend do dado tratado no array auxiliar.

In [12]:
anime.genre[0:5] # Uma amostra pré tratamento.

0            [Drama,  Romance,  School,  Supernatural]
1    [Action,  Adventure,  Drama,  Fantasy,  Magic,...
2    [Action,  Comedy,  Historical,  Parody,  Samur...
3                                  [Sci-Fi,  Thriller]
4    [Action,  Comedy,  Historical,  Parody,  Samur...
Name: genre, dtype: object

In [13]:
anime['genre'] = genre_lista # subtiuimos os dados problemáticos com outro tratado.

In [14]:
anime.genre[0:5] # Uma amostra pós tratamento.

0                    Drama,Romance,School,Supernatural
1    Action,Adventure,Drama,Fantasy,Magic,Military,...
2    Action,Comedy,Historical,Parody,Samurai,Sci-Fi...
3                                      Sci-Fi,Thriller
4    Action,Comedy,Historical,Parody,Samurai,Sci-Fi...
Name: genre, dtype: object

In [15]:
one_hot_genre = anime['genre'].str.get_dummies(',') # Realiza o one hot encoding da coluna genre.

In [16]:
one_hot_genre.columns

Index(['Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama',
       'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical', 'Horror',
       'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music',
       'Mystery', 'Parody', 'Police', 'Psychological', 'Romance', 'Samurai',
       'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai', 'Shounen',
       'Shounen Ai', 'Slice of Life', 'Space', 'Sports', 'Super Power',
       'Supernatural', 'Thriller', 'Vampire', 'Yaoi', 'Yuri'],
      dtype='object')

Observamos uma grande redução do numero de colunas que iriam para o modelo, exatamente a metade. Com isso podemos criar um modelo de recomendação mais rápido e com menor tamanho, importante para escabilidade da ferramenta

In [17]:
one_hot_type = anime['type'].str.get_dummies() # Realiza o one hot encoding da coluna type.
anime_one_hot = pd.concat([anime, one_hot_genre,one_hot_type], axis=1) #está concatenando os dataframes da base de anime, one_hot_genre e one_hot_type.

In [18]:
one_hot_genre

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Quando rodamos o modelo, havia indicação de valores não *int* na coluna **episodes**, portanto, era uma inforamação que a base não sabia. Nessa caso inputamos valor 0 em *'Unknown'*

In [19]:
anime_one_hot[anime_one_hot['episodes'] == 'Unknown'] = 1 

Abaixo realizamos exclusão de colunas que não estavam rendendo bons resultados empiricos, sempre tendenciando os resultados, ou que não seriam importantes para a construção do Modelo. 

Portanto, as colunas  **members**,  **type** e  **episodes** causam a tendência e foram apagadas.

As colunas **name** não irá agregar pois é o nome de cada anime e não possui algum valor, a coluna  **genre ** crua já foi tratada e transformada em *one hot encoding* e será excluída.

A coluna *anime_id* se tornará o index para melhor filtro e localização de dados nas inferências futuras.

In [20]:
anime_one_hot.set_index('anime_id', inplace=True) # transforma a coluna anime_id em index do dataset anime_one_hot.
del anime_one_hot['name']
del anime_one_hot['genre']
del anime_one_hot['type']
del anime_one_hot['members']
del anime_one_hot['episodes']

Base de treino está finalizada abaixo. Possui 1207 registros e 50 Colunas

In [21]:
anime_one_hot

,rating,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,...,Thriller,Vampire,Yaoi,Yuri,Movie,Music,ONA,OVA,Special,TV
anime_id,,,,,,,,,,,,,,,,,,,,,
32281,9.37,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
5114,9.26,1,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
28977,9.25,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9253,9.17,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
9969,9.16,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9316,4.15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5543,4.28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5621,4.88,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## Modelo de Recomendação

In [22]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(anime_one_hot) # instanciando o objeto do NearestNeighbors e realizando o Fit do modelo

In [23]:
anime_one_hot.shape

(12017, 50)

Vamos criar uma inferência, nesse caso vamos sugerir um anime e o modelo irá recomendar 5 animes mais próximos do sugerido

Temos que pegar um indice que aponta para um anime especifico, coletar os valores correspondentes.  

In [24]:
teste = np.array(anime_one_hot.loc[11061]).reshape(1,50) #Buscando o anime pelo index do dataset anime_one_hot

In [25]:
recomendados = nbrs.kneighbors(teste, return_distance=False).flatten() # Realiza a inferência e aplicando o flattem para trasnforar o output em um array 1D

In [26]:
anime[anime['anime_id'] == 11061] # O que anime foi infêrido

,anime_id,name,genre,type,episodes,rating,members
6,11061,Hunter x Hunter (2011),"Action,Adventure,Shounen,Super Power",TV,148,9.13,425855


Logo abaixo temos os 5 animes mais próximo segundo o modelo KNN, que podemos analizar que alguns são realtivos ao titulo pesquisado, Hunter x Hunter.

In [27]:
anime[anime['anime_id'].isin(anime_one_hot.index.values[recomendados])].iloc[1:] #Filtramos com os index sugeridos no modelo de recomendação na base anime para ter uma informação geral

,anime_id,name,genre,type,episodes,rating,members
112,136,Hunter x Hunter,"Action,Adventure,Shounen,Super Power",TV,62,8.48,166255
113,30503,Noragami Aragoto,"Action,Adventure,Shounen,Supernatural",TV,13,8.48,299434
145,137,Hunter x Hunter OVA,"Action,Adventure,Shounen,Super Power",OVA,8,8.41,53168
146,139,Hunter x Hunter: Greed Island Final,"Action,Adventure,Shounen,Super Power",OVA,14,8.41,55787
175,1604,Katekyo Hitman Reborn!,"Action,Comedy,Shounen,Super Power",TV,203,8.37,258103


Para facilitar essa inferência vamos cosntruir uma função que realize essa tarefa em um único comando:

In [28]:
def recomendacao(index):
    loaded_model = pickle.load(open('recomendacao_model.sav', 'rb')) # Realiza o Loading dos pesos do modelo
    teste2 = np.array(anime_one_hot.loc[index]).reshape(1,50) # busca o input conforme o index para a inferência
    recomendados2 = loaded_model.kneighbors(teste2, return_distance=False).flatten() # Realiza a inferência e aplicando o flattem para trasnforar o output em um array 1D
    temp = anime[anime['anime_id'].isin(anime_one_hot.index.values[recomendados2])] #Filtra cos index sugeridos pelo modelo de recomendação na base anime para ter uma informação geral
    # normalmente em primeiro lugar da lista sera o anime que foi inputado, pois ele é totalmente semelhando com um ponto presente no modelo, nesse caso fitlramos tudo contrario ao index inferido
    temp = temp[temp['anime_id'] != index] 
    return temp #retorna uma pequena tabela em DataFrame

In [29]:
usuario

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [30]:
usuario[usuario['user_id'] ==100]

,user_id,anime_id,rating
8273,100,1281,10
8274,100,6746,10
8275,100,8074,9
8276,100,9919,10
8277,100,11757,10


In [31]:
recomendacao(1281)

,anime_id,name,genre,type,episodes,rating,members
870,228,Jigoku Shoujo,"Horror,Mystery,Psychological,Supernatural",TV,26,7.79,172415
1481,10798,UN-GO,"Mystery,Supernatural",TV,11,7.53,84499
1684,7662,Shinrei Tantei Yakumo,"Horror,Mystery,Shoujo,Supernatural",TV,13,7.47,78952
1910,326,Petshop of Horrors,"Horror,Josei,Mystery,Supernatural",TV,4,7.41,32696
1975,6980,Kaidan Restaurant,"Horror,Kids,Mystery,Supernatural",TV,23,7.39,9994


In [32]:
recomendacao(6746)

,anime_id,name,genre,type,episodes,rating,members
161,32867,Bungou Stray Dogs 2nd Season,"Mystery,Seinen,Supernatural",TV,12,8.39,83641
275,27833,Durarara!!x2 Ketsu,"Action,Mystery,Supernatural",TV,12,8.23,115295
360,23199,Durarara!!x2 Shou,"Action,Mystery,Supernatural",TV,12,8.15,189407
391,27831,Durarara!!x2 Ten,"Action,Mystery,Supernatural",TV,12,8.12,132506
1481,10798,UN-GO,"Mystery,Supernatural",TV,11,7.53,84499


In [33]:
recomendacao(8074)

,anime_id,name,genre,type,episodes,rating,members
1765,5034,Shikabane Hime: Kuro,"Action,Horror",TV,12,7.45,46709
3192,2404,Zombie-Loan,"Action,Horror,Shounen,Supernatural",TV,11,7.06,93637
3372,3710,Hakaba Kitarou,"Horror,Supernatural",TV,11,7.01,6649
3985,19855,Nobunagun,"Action,Supernatural",TV,13,6.83,43551
5095,2252,Devilman,"Action,Demons,Horror,Supernatural",TV,39,6.55,5532


## Salvar Modelo

In [35]:
filename = 'recomendacao_model.sav' # Nome para o aqruivo que guardará os pesos do nosso modelo de Recomendação.
pickle.dump(nbrs, open(filename, 'wb')) # Salvando o modelo de Recomendação.

## Sistema de Recomendação em Produção

*bokeh serve --show webapp.py*